## Calculating the Riemann Sum Using Elevation Data

This script simulates calculating the Riemann Sum across a digital elevation model (DEM) using the ArcPy library. For the sake of convenience, I have created a small test raster dataset which I'll use as the DEM. 

## Creating Test Data

We can use the example to generate a random raster from the ArcGIS Pro [website](https://pro.arcgis.com/en/pro-app/tool-reference/data-management/create-random-raster.htm). First I want to create a test raster and some random points on which I'll create buffers. These buffers will be the areas on which I'll perform the Riemann sum. I was able to use `Zonal Statistics as Table` to create a table afterwhich I can transfer the resulting sums to the shapefile attribute table.  As for dealing with multiple rasters, maybe I can create a `for` loop to handle multiple raster and polygon files. For now, I am only using one raster file and one shapefile. 


In [ ]:
# name: arojas
# date: 04/09/2020
# file: riemann_arcpy
# tests riemann sum on sample raster using arcpy functionality

import arcpy
 
# define variables
arcpy.env.workspace = "T:\\Students\\alfredoj\\final_project\\output"
arcpy.env.overwriteOutput = True # overwrite output

# out_path = "output"
outFile = "test.tif"
distribution = "POISSON 6.4"
outExtent = "250 250 750 750"
cellSize = 10
 
# Execute CreateRandomRaster
arcpy.CreateRandomRaster_management(arcpy.env.workspace, outFile, distribution, outExtent, cellSize)

# Create random points in the features of a constraining feature class
points = "random_points.shp"
con_extent = "test.tif" # may not need the absolute path
arcpy.CreateRandomPoints_management(arcpy.env.workspace, points, "", con_extent, 10)

# input/output data
input_ras = "test.tif"
input_pts = "random_points.shp"
output_ras = "test_Proj2.tif"
output_pts = "pts_Proj2.shp"

# create a spatial reference object for the output coordinate system
out_CRS = "PROJCS['NAD_1983_UTM_Zone_1N',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['False_Easting',500000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-177.0],PARAMETER['Scale_Factor',0.9996],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]"

# run the projection tools
arcpy.ProjectRaster_management(input_ras, output_ras, out_CRS, "NEAREST", "", "", "", out_CRS)
arcpy.Project_management(input_pts, output_pts, out_CRS, "", out_CRS, "NO_PRESERVE_SHAPE", "", "NO_VERTICAL")


## Riemann Sum

Now that I have my test data using ArcPy methods, I can create "mounds" (i.e., buffers around points) and calculate the Riemann Sums using ArcPy geoprocessing workflow. The thing that I learned most about this process is that you are only allowed to access a cursor once unless you reset it. When I start working on the really large datasets, I can see how this code works, hopefully making only slight modifications.

In [ ]:
# Crate buffer around test points
out_buff = "pt_buffer.shp"
arcpy.Buffer_analysis(points, out_buff, "30 Meters", "FULL", "ROUND", "NONE")

# see fields
fields = arcpy.ListFields(out_buff)
for field in fields:
    print("{0} is a type of {1} with a length of {2}".format(field.name, field.type, field.length))

from arcpy.sa import *
# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

# define parameters
input_data = "pt_buffer.shp"
zoneField = "FID"
valueRaster = "test.tif"
outTable = "test.dbf"

# The following inputs are layers or table views: "pt_buffer", "test.tif"
ZonalStatisticsAsTable(input_data, zoneField, valueRaster, outTable, "DATA", "SUM")

# look at fields
fields = arcpy.ListFields("test.dbf")
for field in fields:
    print("{0} is a type of {1} with a length of {2}".format(field.name, field.type, field.length))

# inspet dbf table to compare initial values to calculations after
var = ['FID_', 'COUNT', 'AREA', 'SUM']
cursor = arcpy.da.SearchCursor("test.dbf", var)
# NOTE: cannot access cursor more than once unless `reset` is called

##for row in cursor:
##    print('{0}, {1}, {2}, {3}'.format(row[0], row[1], row[2], row[3])) 

# check if field already exists
if findField(fields, "R_SUM"):
    print("R_SUM field already exists, deleting. . .")
    arcpy.DeleteField_management(input_shp, ["R_SUM"])

# get raster cell size, make sure to square
ras_value = arcpy.GetRasterProperties_management("test.tif", "CELLSIZEX")

# stores values in list
r_list = []
for row in cursor:
    r_sum = row[3] * int(ras_value[0])**2
    r_list.append(r_sum)
    
# add field in buffer attr table
arcpy.AddField_management(out_buff, "R_SUM", "FLOAT")

# update out_buff, multiply cursor by ras_value^2, and insert in rowUpdate
update_row = arcpy.da.UpdateCursor(out_buff, "R_SUM")
count = 0
for row in update_row:
    row[0] = r_list[count]
    update_row.updateRow(row) #
    count = count + 1

# compare R_SUM to SUM above
inspect = arcpy.da.SearchCursor(out_buff, "R_SUM")
for field in inspect:
    print("{0}".format(field[0]))

del(cursor, update_row, row, inspect, field)